In [ ]:
import tensorflow as tf
import pickle
from pathlib import Path
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Paths
save_dir = Path("/content/drive/MyDrive/EmotionDetection_Fixed")
save_dir.mkdir(parents=True, exist_ok=True)

# ---- Load metadata ----
meta_path = "/content/drive/MyDrive/EmotionDetection_Trained/EfficientNetB0_metadata.pkl"
with open(meta_path, "rb") as f:
    meta = pickle.load(f)

num_classes = len(meta["emotion_labels"])
print(f"Detected emotion classes: {meta['emotion_labels']}")

# ---- Rebuild EfficientNet for grayscale input ----
inp = Input(shape=(225, 225, 1))  # grayscale input
x = tf.keras.layers.Conv2D(3, (3, 3), padding="same")(inp)  # map grayscale → 3 channels

# ⚠️ IMPORTANT: use weights=None (not imagenet)
base_model = EfficientNetB0(weights=None, include_top=False, input_tensor=x)

# Freeze base model (optional — you can unfreeze for fine-tuning)
for layer in base_model.layers:
    layer.trainable = False

# Classification head
h = GlobalAveragePooling2D()(base_model.output)
h = Dense(128, activation="relu")(h)
h = Dropout(0.4)(h)
out = Dense(num_classes, activation="softmax")(h)

model = Model(inp, out, name="EfficientNetB0_fixed_gray")
print("✅ Model built successfully!")
print("Input shape:", model.input_shape)

# ---- Save fixed model ----
model_path = save_dir / "EfficientNetB0_emotion_model.keras"
model.save(model_path)
print(f"💾 Saved fixed EfficientNetB0 (grayscale-compatible): {model_path}")

# ---- Save metadata ----
meta_save = save_dir / "EfficientNetB0_metadata.pkl"
with open(meta_save, "wb") as f:
    pickle.dump(meta, f)
print(f"🧾 Saved metadata: {meta_save}")


Detected emotion classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
✅ Model built successfully!
Input shape: (None, 225, 225, 1)
💾 Saved fixed EfficientNetB0 (grayscale-compatible): /content/drive/MyDrive/EmotionDetection_Fixed/EfficientNetB0_emotion_model.keras
🧾 Saved metadata: /content/drive/MyDrive/EmotionDetection_Fixed/EfficientNetB0_metadata.pkl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')